# Monitoring IP Addresses with Python
This script is a powerful tool for monitoring the status of specific IP addresses. It uses Python, a high-level, interpreted programming language known for its readability and versatility. The script also leverages several libraries to perform its tasks, including sqlmodel, subprocess, platform, selenium, and time.

## Benefits of the Script
This script is beneficial for network administrators who need to monitor the status of IP addresses. It automates the process of pinging IP addresses and provides real-time updates on their status. By sending a WhatsApp message when an IP address goes offline, the script ensures that network issues can be addressed promptly. Furthermore, by using a database to track the status of IP addresses, the script provides a historical record of network uptime and downtime.

In conclusion, this script is a great example of how Python and its libraries can be used to automate network monitoring tasks. It showcases the power of Python in system and network administration.

## How the Script Works
The script continuously pings a list of IP addresses. If an IP address responds to the ping, it is considered online. If it doesn’t respond, it is considered offline. The script performs different actions based on the status of the IP address:

If an IP address is online: The script checks if the IP address is already in the database. If it is, the script removes it from the database.
If an IP address is offline: The script checks if the IP address is already in the database. If it isn’t, the script adds it to the database and sends a WhatsApp message notifying a user that the IP address is not responding to pings.

## Libraries Used
### sqlmodel: 
This library is used to interact with a PostgreSQL database. It allows the script to create, read, update, and  delete entries in the database.
### subprocess: 
This library is used to execute the ping command in the system’s command line.
### platform:
 This library is used to determine the operating system of the host machine.
### selenium:
 This library is used to automate browser actions, specifically sending a WhatsApp message.
### time:
 This library is used to control the script’s execution speed.

These lines import the necessary libraries. sqlmodel is used for database operations, subprocess and platform are used for pinging IP addresses, and selenium is used for automating WhatsApp messages.

In [1]:
from typing import Optional
from sqlmodel import SQLModel, Field, create_engine,Session
from sqlmodel import select
import subprocess
import platform
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from dotenv import load_dotenv
import os


# Database connection string

In [4]:
load_dotenv()
database_connection_str = os.getenv('db')   

# Create engine for database connection
#echo true only development env not for production
engine = create_engine(database_connection_str, echo=False)    

# Define Server class with SQLModel
class Server(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    server_ip: str

# Function to create database and tables
def create_db_and_tables():
    SQLModel.metadata.create_all(engine)

InvalidRequestError: Table 'server' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

# Create database and tables

In [5]:
create_db_and_tables()

# Function to create a server with a given host

In [6]:
def create_server(host:str):
    server1 = Server(server_ip=host)
    session = Session(engine)
    session.add(server1)
    session.commit()
    session.close()

# Function to delete an IP from the database

In [7]:
def delete_ip(host:str):
    with Session(engine) as session:
        statement = select(Server).where(Server.server_ip == host)
        results = session.exec(statement)
        ip_address = results.one()
        print("IP Address: ", ip_address)

        session.delete(ip_address)
        session.commit()

        print("Deleted ip:", ip_address)

# Function to select an IP from the database

In [8]:
def select_ip(host:str):
    data = []
    with Session(engine) as session:
        statement = select(Server).where(Server.server_ip == host)
        results = session.exec(statement)
        for ip,name in results:
            new = data.append(ip)
        return data    

#### Note: This code is for individual person messages. If you want to send a message in a group, please refer to the last code.

In [7]:
# Name of the cookie file for WhatsApp
whatsapp_file_cookie_name = ""

# Path to the Chrome profile
chrome_profile_path = "user-data-dir=C:/Users/Rafi Ali/AppData/Local/Google/Chrome/User Data/Default/"

# Function to ping a host
def ping(host):
    """
    Returns True if the host responds to a ping request.
    """
    param = '-n' if platform.system().lower() == 'windows' else '-c'
    command = ['ping', param, '1', host]
    return subprocess.call(command) == 0

# List of hostnames to ping
hostname = ["10.3.10.50", "10.3.10.97","10.3.21.201"]

# Main loop to continuously ping hosts
while True:
    for host in hostname:
        if ping(host):
            print(f"{host} is online")
            ifServer = select_ip(host)
            for name,ip in ifServer:
                db_ip = ip
                if db_ip == host:
                    print("already exist")
                    delete_ip(host)
                        
        else:
            print(f"{host} is offline")
            ip = ""
            ifServer = select_ip(host)
            for name,ip in ifServer:
                db_ip = ip
                if db_ip == host:
                    print("already exist")        
            if host == ip:
                print("Already exist in db")                
            else:
                create_server(host)
                # time.sleep(5)
                whatsapp_file_cookie_name = ""

                chrome_profile_path = "user-data-dir=C:/Users/Rafi Ali/AppData/Local/Google/Chrome/User Data/Default/"

                chrome_options = webdriver.ChromeOptions()
                chrome_options.add_argument(chrome_profile_path)
                driver = webdriver.Chrome(options=chrome_options)
                driver.get(f"https://web.whatsapp.com/send?phone=+9203190269909&text=This IP address {host} is not responding to pings")
                print("hello")
                time.sleep(30)
                # Use find_element instead of find_elements
                element = driver.find_element(by=By.XPATH, value="/html/body/div[1]/div/div[2]/div[4]/div/footer/div[1]/div/span[2]/div/div[2]/div[2]/button")
                element.click()  # Now you can call .click() on the single element
                time.sleep(10)
                driver.quit()
    time.sleep(20)  # Wait for 20 minute before checking again        


10.3.10.50 is online
10.3.10.97 is online
10.3.21.201 is offline
hello


#### Note: This code is for group  messages. If you want to send a individual person message, please refer to the upper code.

In [10]:
# Name of the cookie file for WhatsApp
whatsapp_file_cookie_name = ""

# Path to the Chrome profile
chrome_profile_path = "user-data-dir=C:/Users/Rafi Ali/AppData/Local/Google/Chrome/User Data/Default/"

# Function to ping a host
def ping(host):
    """
    Returns True if the host responds to a ping request.
    """
    param = '-n' if platform.system().lower() == 'windows' else '-c'
    command = ['ping', param, '1', host]
    return subprocess.call(command) == 0

# List of hostnames to ping
hostname = ["10.3.10.50", "10.3.10.97","10.3.21.201","10.3.10.222","10.3.10.18","10.3.10.30"]

# Main loop to continuously ping hosts
while True:
    for host in hostname:
        if ping(host):
            print(f"{host} is online")
            ifServer = select_ip(host)
            for name,ip in ifServer:
                db_ip = ip
                if db_ip == host:
                    print("already exist")
                    delete_ip(host)
                        
        else:
            print(f"{host} is offline")
            ip = ""
            ifServer = select_ip(host)
            for name,ip in ifServer:
                db_ip = ip
                if db_ip == host:
                    print("already exist")        
            if host == ip:
                print("Already exist in db")                
            else:
                create_server(host)
                # time.sleep(5)
                whatsapp_file_cookie_name = ""

                chrome_profile_path = "user-data-dir=C:/Users/Rafi Ali/AppData/Local/Google/Chrome/User Data/Default/"

                chrome_options = webdriver.ChromeOptions()
                chrome_options.add_argument(chrome_profile_path)
                driver = webdriver.Chrome(options=chrome_options)
                driver.get("https://web.whatsapp.com/accept?code=BaRxMtgOLVx5vDPIOMjjkt")
                print("hello from else")
                time.sleep(30)
                # Clear field to empty it from any previous data
                driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[4]/div/footer/div[1]/div/span[2]/div/div[2]/div[1]/div[2]/div[1]").clear()

                # Enter Text
                driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[4]/div/footer/div[1]/div/span[2]/div/div[2]/div[1]/div[2]/div[1]").send_keys(f"This IP address {host} is not responding to pings" )
                # Use find_element instead of find_elements
                element = driver.find_element(by=By.XPATH, value="/html/body/div[1]/div/div[2]/div[4]/div/footer/div[1]/div/span[2]/div/div[2]/div[2]/button")
                element.click()  # Now you can call .click() on the single element
                time.sleep(10)
                driver.quit()
    time.sleep(20)  # Wait for 20 minute before checking again    

10.3.10.50 is online
10.3.10.97 is online
10.3.21.201 is offline
already exist
Already exist in db
10.3.10.222 is online
10.3.10.18 is online
already exist
IP Address:  server_ip='10.3.10.18' id=16
Deleted ip: server_ip='10.3.10.18' id=16
10.3.10.30 is online
10.3.10.50 is offline
hello from else
10.3.10.97 is online
10.3.21.201 is offline
already exist
Already exist in db
10.3.10.222 is online
10.3.10.18 is online
10.3.10.30 is online
10.3.10.50 is offline
already exist
Already exist in db
10.3.10.97 is online
10.3.21.201 is offline
already exist
Already exist in db
10.3.10.222 is online
10.3.10.18 is online
10.3.10.30 is online
10.3.10.50 is online
already exist
IP Address:  server_ip='10.3.10.50' id=17
Deleted ip: server_ip='10.3.10.50' id=17
10.3.10.97 is online
10.3.21.201 is offline
already exist
Already exist in db
10.3.10.222 is online
10.3.10.18 is online
10.3.10.30 is offline
hello from else
10.3.10.50 is online
10.3.10.97 is online
10.3.21.201 is offline
already exist
Alread

  <a href="https://www.linkedin.com/in/therafiali/"><img  alt="LinkedIn" title="LinkedIn" src="https://img.shields.io/badge/LinkedIn-0b5fbb?style=for-the-badge&logo=linkedin&logoColor=white"/></a>